In [ ]:
import pandas as pd
matchData=pd.read_csv("matchedVector_Oct.csv")

matchData

In [ ]:
#initialize language model
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

japanese_sentences = ["犬", "子犬はいいです", "私は犬と一緒にビーチを散歩するのが好きです"]
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
ja_result = embed(japanese_sentences)
ja_result

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#select features (include everything but text)
user_X=matchData.loc[:,"gender_公開したくない_user":"coping_strategy_飲酒_user"]
target_X=matchData.loc[:,"gender_公開したくない_matched":"coping_strategy_飲酒_matched"]
user_X_Id=matchData.loc[:,["id_user"]]
matched_X_Id=matchData.loc[:,["id_matched"]]
isMatched=matchData.loc[:,["isMatched"]]


#vectorize the languages
loss_description_user=matchData.loc[:,"loss_description_user"]
loss_vector_col_name = ["loss_user_"+str(num) for num in range(0, 512)]
loss_vector = pd.DataFrame(embed(loss_description_user).numpy(),columns=loss_vector_col_name)

loss_description_matched=matchData.loc[:,"loss_description_matched"]
loss_vector_match_col_name = ["loss_user_matched_"+str(num) for num in range(0, 512)]
loss_vector_matched = pd.DataFrame(embed(loss_description_matched).numpy(),columns=loss_vector_match_col_name)

hobbies_description=matchData.loc[:,"hobbies_description_user"]
hobbies_col_name = ["hobbies_description_"+str(num) for num in range(0, 512)]
hobbies_vector = pd.DataFrame(embed(hobbies_description).numpy(),columns=hobbies_col_name)

hobbies_description_matched=matchData.loc[:,"hobbies_description_matched"]
hobbies_col_name_matched = ["hobbies_description_matched_"+str(num) for num in range(0, 512)]
hobbies_vector_matched = pd.DataFrame(embed(hobbies_description_matched).numpy(),columns=hobbies_col_name_matched)


matchEncodedData=pd.DataFrame(pd.concat([
    pd.DataFrame(user_X,columns=user_X.columns),
    pd.DataFrame(target_X,columns=target_X.columns),
    pd.DataFrame(user_X_Id,columns=user_X_Id.columns),
    pd.DataFrame(matched_X_Id,columns=matched_X_Id.columns),
    loss_vector,loss_vector_matched,hobbies_vector,hobbies_vector_matched,
    pd.DataFrame(isMatched,columns=isMatched.columns)

]
    ,axis=1))

# user_features=pd.DataFrame(pd.concat([
#     pd.DataFrame(user_X,columns=user_X.columns),
#     loss_vector,hobbies_vector
# ]
#     ,axis=1))

# item_features=pd.DataFrame(pd.concat([
#     pd.DataFrame(target_X,columns=target_X.columns),
#     loss_vector_matched,hobbies_vector_matched,
# ]
#     ,axis=1))

# interaction_matrix=pd.DataFrame(pd.concat([
#     pd.DataFrame(user_X_Id,columns=user_X_Id.columns),
#     pd.DataFrame(match_X_Id,columns=match_X_Id.columns),
#     pd.DataFrame(isMatched,columns=isMatched.columns)
# ]
#     ,axis=1))

In [ ]:
import random

def getExistingTrainingToTest(train_index,test_index,ratio,numMatches):
    pickNum=round(numMatches*ratio)
    numUsers=round(len(test_index)/numMatches)
    index=0
    new_train_index=[]
    new_test_index=[]
    moveList=[]
    for index in range(numUsers):
        randomIndexList=random.sample(range(numMatches), pickNum)
        randomIndexList=[x+(index*numMatches) for x in randomIndexList]
        moveList = moveList+[test_index[x] for x in randomIndexList]
        index=index+1
    #move the numbers in the move list to the test and train index
    print(len(moveList))
    new_test_index=list(filter(lambda x: not(x in moveList),test_index))
    new_train_index=list(train_index)
    move_index=moveList
    return new_train_index,move_index,new_test_index

In [ ]:
from collections import defaultdict

from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold
from surprise import Reader
from surprise import accuracy

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
        
    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        
        # normalize from zero to one
        getNormalizeNumber=[]
        for eachRating in user_ratings:
            getNormalizeNumber.append(eachRating[0])
            
        normalized=NormalizeData(getNormalizeNumber)
        
        newRating=[]
        for index,eachRating in enumerate(user_ratings):
            newRating.append((normalized[index],eachRating[1]))
        user_ratings=newRating
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        
        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        
        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

        if n_rel == 0:
            recalls[uid] = -1
            precisions[uid]=-1
        
    return precisions, recalls

In [ ]:
#for evaluation 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import LeaveOneGroupOut

from sklearn import preprocessing

from surprise import Dataset
from surprise import Reader
from surprise import SVDpp,BaselineOnly,SlopeOne
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

import numpy as np

#ignore all warnings
import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

from statistics import mean

#threashold not used
def testEverythingGroupCF(X,Y,groupString,k_value,ratio):
    group=X[groupString].values
    
    skf = LeaveOneGroupOut()
    skf.get_n_splits(X,Y,group)
    print(skf)

    cf_p_k=[]
    cf_r_k=[]
    
    base_p_k=[]
    base_r_k=[]
    
    slope_p_k=[]
    slope_r_k=[]
    
    for train_index, test_index in skf.split(X,Y,group):
        train_index,move_index,test_index=getExistingTrainingToTest(train_index,test_index,ratio,51)
        
        X_train = X[X.index.isin(train_index)]
        X_train_moved=X[X.index.isin(move_index)]
        X_test = X[X.index.isin(test_index)]

        Y_train = Y[Y.index.isin(train_index)]
        Y_train_moved=Y[Y.index.isin(move_index)]
        Y_test = Y[Y.index.isin(test_index)]

        X_train=pd.DataFrame(pd.concat([
           pd.DataFrame(X_train),
           pd.DataFrame(X_train_moved)
        ]))
        
        Y_train=pd.DataFrame(pd.concat([
           pd.DataFrame(Y_train),
           pd.DataFrame(Y_train_moved)
        ]))
        
        
        #build the dataset with the user ID & match ID, to show the unique features for lightfm
        reader = Reader(rating_scale=(0, 1))
        data_train = Dataset.load_from_df(X_train[['id_user', 'id_matched', 'isMatched']], reader)
        data_test = Dataset.load_from_df(X_test[['id_user', 'id_matched', 'isMatched']], reader)
        testset = [data_test.df.iloc[i].to_list() for i in range(len(data_test.df))]
        
        ############DO SVD ######################
        param_grid = {'n_epochs': [5,20,100], 'reg_all': [0.4,0.8]}
        gs = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3)
        gs.fit(data_train)
        print(gs)
        algo = gs.best_estimator['rmse']
        algo.fit(data_train.build_full_trainset())
        
        pred= algo.test(testset)
        
        precisions, recalls = precision_recall_at_k(pred, k=k_value, threshold=0.5)
        print(sum(prec for prec in precisions.values()) / len(precisions))
        print(sum(rec for rec in recalls.values()) / len(recalls))        
        cf_p_k.append(sum(prec for prec in precisions.values()) / len(precisions))
        cf_r_k.append(sum(rec for rec in recalls.values()) / len(recalls))
        
        ############DO BASELINE ######################
        algo = BaselineOnly()
        algo.fit(data_train.build_full_trainset())
        pred= algo.test(testset)
        
        precisions, recalls = precision_recall_at_k(pred, k=k_value, threshold=0.5)
        print(sum(prec for prec in precisions.values()) / len(precisions))
        print(sum(rec for rec in recalls.values()) / len(recalls))        
        base_p_k.append(sum(prec for prec in precisions.values()) / len(precisions))
        base_r_k.append(sum(rec for rec in recalls.values()) / len(recalls)) 
        
        ############DO SLOPEONE ######################
        algo = SlopeOne()
        algo.fit(data_train.build_full_trainset())
        pred= algo.test(testset)
        
#         #get the threashold 
#         est_value=[]
#         for eachItem in pred:
#             est_value.append(eachItem.est)
#         est_value_np = np.array(est_value)
#         adj_threashold = np.percentile(est_value_np, 80)
#         print("threadshold:"+str(adj_threashold))
        
        precisions, recalls = precision_recall_at_k(pred, k=k_value, threshold=0.5)
        print(sum(prec for prec in precisions.values()) / len(precisions))
        print(sum(rec for rec in recalls.values()) / len(recalls))        
        slope_p_k.append(sum(prec for prec in precisions.values()) / len(precisions))
        slope_r_k.append(sum(rec for rec in recalls.values()) / len(recalls))  
        
    print("SVD")
    print(cf_p_k)
    print(mean([x for x in cf_p_k  if x != -1]))
    print(cf_r_k)
    print(mean([x for x in cf_r_k  if x != -1]))
    print("BASELINE")
    print(base_p_k)
    print(mean([x for x in base_p_k  if x != -1]))
    print(base_r_k)
    print(mean([x for x in base_r_k  if x != -1]))
    print("SLOPE")
    print(slope_p_k)
    print(mean([x for x in slope_p_k  if x != -1]))
    print(slope_r_k)
    print(mean([x for x in slope_r_k  if x != -1]))

In [ ]:
X=matchEncodedData
Y=isMatched

testEverythingGroupCF(X,Y,"id_user",5,0.5)

In [ ]:
testEverythingGroupCF(X,Y,"id_user",5,0.25)

In [ ]:
testEverythingGroupCF(X,Y,"id_user",5,0.1)

In [ ]:
testEverythingGroupCF(X,Y,"id_user",5,0)

In [ ]:
testEverythingGroupCF(X,Y,"id_user",5,0.75)

In [ ]:
testEverythingGroupCF(X,Y,"id_user",10,0)

In [ ]:
testEverythingGroupCF(X,Y,"id_user",10,0.1)